In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin

class StackingEnsembleCV(BaseEstimator, ClassifierMixin):
    # [Previous implementation remains the same]
    pass  # keeping previous implementation

def grid_search_stacking(df, target_cols):
    # Prepare data as before
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    
    # Encode categorical variables
    le = LabelEncoder()
    categorical_columns = train_df.select_dtypes(include=['object']).columns
    
    train_encoded = train_df.copy()
    test_encoded = test_df.copy()
    
    for column in categorical_columns:
        le = LabelEncoder()
        train_encoded[column] = le.fit_transform(train_encoded[column].astype(str))
        test_encoded[column] = le.transform(test_encoded[column].astype(str))

    X_train = train_encoded.drop(columns=target_cols, errors='ignore')
    y_train = train_encoded['target'].astype(int)
    X_test = test_encoded.drop(columns=target_cols, errors='ignore')
    y_test = test_encoded['target'].astype(int)

    # Define parameter grid for each model
    param_grid = {
        'n_folds': [3, 5],
        'base_models__gb__n_estimators': [100, 170, 200],
        'base_models__gb__learning_rate': [0.1, 0.15, 0.2],
        'base_models__gb__min_samples_split': [2, 5, 10],
        'base_models__gb__min_samples_leaf': [2, 5, 10],
        
        'base_models__rf__n_estimators': [100, 200, 300],
        'base_models__rf__max_depth': [8, 10, 12],
        'base_models__rf__min_samples_split': [2, 4, 6],
        'base_models__rf__min_samples_leaf': [2, 4, 6],
        
        'base_models__xgb__n_estimators': [100, 150, 200],
        'base_models__xgb__learning_rate': [0.08, 0.12, 0.15],
        'base_models__xgb__max_depth': [4, 6, 8],
        
        'base_models__lgbm__n_estimators': [100, 150, 200],
        'base_models__lgbm__learning_rate': [0.08, 0.12, 0.15],
        'base_models__lgbm__max_depth': [4, 6, 8],
        
        'meta_model__C': [0.1, 0.5, 1.0],
        'meta_model__max_iter': [600, 800, 1000]
    }

    # Initialize GridSearchCV
    grid_search = GridSearchCV(
        estimator=StackingEnsembleCV(),
        param_grid=param_grid,
        cv=5,
        scoring='roc_auc',
        n_jobs=-1,
        verbose=2
    )

    # Fit grid search
    print("Starting Grid Search...")
    grid_search.fit(X_train, y_train)

    # Get best model
    best_model = grid_search.best_estimator_

    # Generate predictions
    predictions = best_model.predict(X_test)
    probabilities = best_model.predict_proba(X_test)[:, 1]

    # Compile results
    results = {
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'cv_results': pd.DataFrame(grid_search.cv_results_),
        'test_predictions': predictions,
        'test_probabilities': probabilities,
        'test_roc_auc': roc_auc_score(y_test, probabilities),
        'test_classification_report': classification_report(y_test, predictions),
        'test_confusion_matrix': confusion_matrix(y_test, predictions)
    }

    return results

# Run grid search
grid_search_results = grid_search_stacking(df_final, target_columns)

